# Import modules

In [1]:
from os.path import abspath, join, pardir
from nilearn import datasets
from roi_mean_cort import roi_mean_interface
from bids.layout import BIDSLayout
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.io import DataSink, DataGrabber
from nipype.algorithms.confounds import TSNR
from nipype.interfaces.utility import Function, IdentityInterface
from nilearn.input_data import NiftiLabelsMasker
from nilearn import plotting
from nipype.interfaces import fsl
from nipype.interfaces.utility import Rename

In [2]:
!ls


C:                        output                    tSNR_ROI_cort.ipynb
README.md                 roi_mean_cort.py          tSNR_ROI_sub.ipynb
Untitled.ipynb            roi_mean_cort.pyc         work_dir_cortical
__pycache__               roi_mean_sub.py           work_dir_subcortical
data                      tSNR_ROI-master
nilearn_cache             tSNR_ROI_cort-Copy1.ipynb


# Define variables for data and atlas

In [3]:
dataset_cort = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
dataset_maps = dataset_cort.maps
project_path = abspath("./data")
print(project_path)
layout = BIDSLayout(project_path)

/Users/marianapereira/Donders/BrainMask_test/tSNR_ROI-master/data


## Show path to output directory

In [4]:
abspath(join(project_path, pardir,"output"))

'/Users/marianapereira/Donders/BrainMask_test/tSNR_ROI-master/output'

### Fetch files by subject_id and run_id
        subject_id and run_id are the plain ids as you get them from `BIDSlayout.get_subjects()`
        
    Parameters
    ---------
    layout: BIDSLayout
        layout to use
    subject_id: str
        SubjectID without sub-
    run_id: str
        RunID without run-
        
    Returns
    -------
    preproc,preprocessed_anat, brainmask, subject_id, run_id

### Def function get_files

In [5]:
def get_files(layout, subject_id, run_id):
    brainmask = layout.get(type="brainmask", return_type="file", subject=subject_id, run=run_id)[0]
    preproc = layout.get(type="preproc", return_type="file", subject=subject_id, run=run_id)
    all_preprocs = layout.get(subject=subject_id, type="preproc", return_type="file", modality="anat")
    print(brainmask)
    print(preproc)
    
    
    return preproc, brainmask, subject_id, run_id

### function resample atlas

In [6]:
def resample_atlas(source_img, template_img):
    from nilearn.image import resample_to_img
    from nilearn import datasets
    dataset_cort = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
    source_img = dataset_cort.maps
    resampled_atlas = resample_to_img(source_img, template_img, interpolation='nearest')
    return resampled_atlas

### function save atlas

In [7]:
def save_atlas(resampled_atlas):
    import os
    from os.path import abspath, join, pardir
    import numpy as np
    import nibabel as nib
    from nilearn.image import load_img
    from os.path import abspath
    work_dir = abspath(join("output")) #"C:/Users/marpere/BrainMask_test/tSNR_ROI-master/output" #abspath(join("output"))
    atlas = load_img(resampled_atlas)
    save_atlas = atlas.to_filename(abspath('resampled_atlas.nii.gz'))
    saved_atlas = abspath('resampled_atlas.nii.gz')
    return saved_atlas

# Get Data

### Setup Node identitysource

In [8]:
identitysource = Node(IdentityInterface(fields=["subject_id", "run_id"]), name="identitysource")

identitysource.iterables= [('subject_id', layout.get_subjects()),
                           ('run_id', layout.get_runs())]

### Setup Node BIDSDataGrabber

In [9]:
BIDSDataGrabber = Node(Function(function=get_files,  input_names=["layout", "subject_id", "run_id",],
                                       output_names=["preproc", "brainmask", "subject_id", "run_id"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.layout = layout

# Processing Nodes

### setup Node for tsnr map calculation

In [10]:
tsnr = Node(TSNR(regress_poly=2), name='tsnr' )

### setup node for atlas resampling

In [11]:
resample_atlas = Node(name='resample_atlas',
               interface=Function(input_names=['template_img', 'source_img'],
                                  output_names=['resampled_atlas'],
                                  function=resample_atlas))
resample_atlas.inputs.source_img = dataset_cort.maps

### setup node for saving resampled atlas (Nifti1image to nii.gz)

In [12]:
save_atlas = Node(name='save_atlas',
               interface=Function(input_names=['resampled_atlas'],
                                  output_names=['saved_atlas'],
                                  function=save_atlas))

### Setup Node ApplyMask

In [13]:
apply_mask = Node(fsl.ApplyMask(), name="apply_mask")

### Set Atlas for mean calculation

In [14]:
roi_mean = Node(roi_mean_interface, name="roi_mean")

### Setup Node roi_mean
#### calculate mean values using function from roi_mean.py

### rename roi_mean.tsv to sub(subject_id)_run(run_id).tsv 

In [15]:
rename = Node(Rename(format_string="sub%(subject_id)s_run%(run_id)02d.tsv"), name='rename')

### Setup Node datasink

In [16]:
datasink = Node(DataSink(), name="datasink")
datasink.inputs.base_directory = abspath(join(project_path, pardir,"output"))

### Connect Nodes, Setup Workflow

In [17]:
tsnr_wf = Workflow(name="TSNR_calculation")
tsnr_wf.base_dir = abspath('work_dir_cortical')
tsnr_wf.connect([(identitysource, BIDSDataGrabber, [('subject_id','subject_id'),
                                                    ('run_id','run_id')]),  
                 (BIDSDataGrabber, resample_atlas, [('preproc', 'template_img')]),
                 (BIDSDataGrabber, tsnr, [('preproc', 'in_file')]),
                 (BIDSDataGrabber, apply_mask, [('brainmask', 'mask_file')]),
                 (resample_atlas, save_atlas, [('resampled_atlas', 'resampled_atlas')]),
                 (save_atlas, apply_mask, [('saved_atlas', 'in_file')]),
                 (apply_mask, roi_mean, [('out_file', 'roi_file')]),
                 
                 (tsnr, roi_mean, [('tsnr_file', 'data_file')]),
                 (roi_mean, rename, [('out_file', 'in_file')]),
                 (BIDSDataGrabber, rename, [('subject_id','subject_id'),
                                                    ('run_id','run_id')]),
                 (tsnr, datasink, [('tsnr_file', 'cortical.@tsnr_map')]),
                 (save_atlas, datasink, [('saved_atlas', 'cortical.@resampled_atlas')]),
                 (apply_mask, datasink, [('out_file', 'cortical.@masked_atlas')]),
                 (rename, datasink, [('out_file', 'cortical.@mean_roi')])]
         )
 

In [18]:
#write_graph(self, dotfilename='graph.dot', graph2use='hierarchical', format='png', simple_form=True)
tsnr_wf.write_graph(dotfilename='/Users/marianapereira/Donders/BrainMask_test/tSNR_ROI-master/work_dir_cortical/TSNR_calculation/graph.dot', 
                    graph2use='colored', format='svg')






190916-10:35:22,593 nipype.workflow INFO:
	 Generated workflow graph: /Users/marianapereira/Donders/BrainMask_test/tSNR_ROI-master/work_dir_cortical/TSNR_calculation/graph.svg (graph2use=colored, simple_form=True).


'/Users/marianapereira/Donders/BrainMask_test/tSNR_ROI-master/work_dir_cortical/TSNR_calculation/graph.svg'

### Run the workflow

In [19]:
tsnr_wf.run()

TypeError: can't pickle _thread._local objects